# **Final Project**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import importlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json



'''
import os
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'



# Ensure this runs before any TensorFlow operations
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print("here")
        print(e)
'''

import glob
import os
import shutil

In [ ]:
'''Imports'''
#from src.preprocessing import preprocess
#from src.training import get_best_model
#from src.validation import Validator
#from src.prediction import Predictor
import src.objects.stack
importlib.reload(src.objects.stack)
from src.objects.stack import Stack

In [ ]:
def remove_directories():
    # List of directories to remove within 'data'
    data_dirs = ['images', 'test', 'train', 'validation']
    base_data_path = 'data'

    # Directory within 'src/models'
    model_dir = 'unet_tuning'
    base_model_path = os.path.join('src', 'models', model_dir)

    # Removing directories in 'data'
    for dir_name in data_dirs:
        dir_path = os.path.join(base_data_path, dir_name)
        shutil.rmtree(dir_path, ignore_errors=True)

    # Removing 'unet_tuning' directory
    shutil.rmtree(base_model_path, ignore_errors=True)
remove_directories()

## **Preprocessing and Augmentation**

In [ ]:
'''Calls Preprocessing functions'''
import src.preprocess.preprocess_list as preprocessing
importlib.reload(preprocessing)

# Set the batch size according to your system's capabilities
BATCH_SIZE = 8
RESOLUTION = (480, 360)

# Call the preprocess function
stack = preprocessing.preprocess(BATCH_SIZE, RESOLUTION)

## **Training**

In [ ]:
'''Calls training functions'''
import src.training as training
importlib.reload(training)
import src.objects.architecture.conv_autoencoder as conv_autoencoder
importlib.reload(conv_autoencoder)


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

stack = training.get_best_model(stack)

## **Prediction**

In [ ]:
'''Calls prediction fucntions in prediction.py and methods from result'''
predictor = Predictor(model_path='path/to/saved/model')
input_data = 'path/to/input/data'
predictions = predictor.predict(input_data)

## **Validation**

In [ ]:
'''Calls the validation fuctions in validation.py and methods from result'''
validator = Validator(model_path='path/to/saved/model')
validation_data = 'path/to/validation/data'
evaluation_metrics = validator.validate(validation_data)